In [134]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [135]:
import pandas as pd
import wandb

api = wandb.Api()

entity = "tunnels-ssl"
project = "01.20"
# Get all runs for the specified project
runs = api.runs(f"{entity}/{project}")


summary_list, config_list, name_list = [], [], []
for run in runs:
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files
    summary_list.append(run.summary._json_dict["test/avg_acc_tag"])

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append({k: v for k, v in run.config.items() if not k.startswith("_")})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame(
    {"avg_acc_tag": summary_list, "config": config_list, "name": name_list}
)
config_df = pd.json_normalize(runs_df["config"])
df = pd.concat([runs_df.drop(columns=["config"]), config_df], axis=1)
df["data.dataset"] = df["data.datasets"].apply(lambda x: x[0])


def get_real_name(row):
    if row["data.exemplars.num_exemplars"] > 0:
        return "replay"
    return row["training.approach.name"]


def is_regularized(row):
    if row["training.vcreg.var_weight"] == 0 and row["training.vcreg.cov_weight"] == 0:
        return False
    return True


df["approach"] = df.apply(get_real_name, axis=1)
df["is_regularized"] = df.apply(is_regularized, axis=1)

In [9]:
import pandas as pd

# Sample data creation
data = {
    "approaches": ["finetuning", "finetuning", "er", "er", "ewc", "ewc"],
    "dataset": ["cifar", "imagenet", "cifar", "imagenet", "cifar", "imagenet"],
    "num_tasks": [5, 10, 10, 20, 20, 10],
}

# Create DataFrame
df = pd.DataFrame(data)

# Create pivot table
pivot_table = pd.pivot_table(
    df,
    index="approaches",
    columns=["dataset", "num_tasks"],
    aggfunc=lambda x: 0,
    fill_value=0,
)
pivot_table

Index(['er', 'ewc', 'finetuning'], dtype='object', name='approaches')

In [138]:
filtered_df = df[
    [
        "avg_acc_tag",
        "approach",
        # "training.vcreg.var_weight",
        # "training.vcreg.cov_weight",
        "misc.seed",
        "data.num_tasks",
        "model.network",
        "data.dataset",
        "is_regularized",
    ]
]
filtered_df = filtered_df.rename(
    columns={
        # "test_acc": "avg_acc_tag",
        "data.num_tasks": "num_tasks",
        "data.dataset": "dataset",
        "model.network": "network",
        "misc.seed": "seed",
    }
)

In [139]:
filtered_df

,avg_acc_tag,approach,seed,num_tasks,network,dataset,is_regularized
0,14.800000,finetuning,0,10,resnet34_skips,imagenet_subset_kaggle,True
1,14.780000,finetuning,2,10,resnet34_skips,imagenet_subset_kaggle,True
2,9.840000,finetuning,0,20,resnet34_skips,imagenet_subset_kaggle,True
3,15.200000,finetuning,1,10,resnet34_skips,imagenet_subset_kaggle,True
4,9.060000,finetuning,1,20,resnet34_skips,imagenet_subset_kaggle,True
...,...,...,...,...,...,...,...
115,28.231579,lwf,0,20,convnext_tiny,imagenet_subset_kaggle,True
116,43.680000,lwf,1,10,convnext_tiny,imagenet_subset_kaggle,True
117,29.389474,lwf,1,20,convnext_tiny,imagenet_subset_kaggle,True
118,42.260000,lwf,2,10,convnext_tiny,imagenet_subset_kaggle,True


In [79]:
import numpy as np

pivot_table = pd.pivot_table(
    df[df["model.network"].str.contains("resnet")],
    index=["approach", "is_regularized"],
    columns=["data.datasets", "data.num_tasks"],
    values="test_acc",
    aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    fill_value=0,
)
pivot_table

KeyError: 'test_acc'

In [140]:
from pathlib import Path

import pandas as pd
import numpy as np

# for csv in Path(
#     "/Users/piotr.hondra/Documents/mgr/cl-adaptation/notebooks/csvs_new/resnet34_skips"
# ).rglob("*.csv"):
#     df = pd.read_csv(csv)
#     df["model.network"] = csv.parts[-2]
#     df.to_csv("all.csv", mode="a", header=False)

In [141]:
all_df = pd.read_csv("all.csv")

In [142]:
df_noreg = all_df[all_df["is_regularized"] == False]

In [143]:
pivot_table = pd.pivot_table(
    df_noreg,
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    fill_value=0,
)
pivot_table

dataset                   cifar100_fixed                          \
num_tasks                             5           10          20   
approach   is_regularized                                          
ewc        False              26.33±3.26  14.57±2.38   6.89±2.62   
finetuning False              23.38±2.88  13.43±1.90   6.61±1.65   
lwf        False              39.98±0.74  27.64±2.30  20.56±3.11   
replay     False              37.58±3.12  32.00±3.79  28.51±3.28   

dataset                   imagenet_subset_kaggle              
num_tasks                                     10          20  
approach   is_regularized                                     
ewc        False                      17.69±3.99   8.91±1.92  
finetuning False                      14.40±2.89   6.72±0.67  
lwf        False                      39.60±4.20  24.61±5.03  
replay     False                      36.57±2.17  32.39±2.94

In [144]:
filtered_df

,avg_acc_tag,approach,seed,num_tasks,network,dataset,is_regularized
0,14.800000,finetuning,0,10,resnet34_skips,imagenet_subset_kaggle,True
1,14.780000,finetuning,2,10,resnet34_skips,imagenet_subset_kaggle,True
2,9.840000,finetuning,0,20,resnet34_skips,imagenet_subset_kaggle,True
3,15.200000,finetuning,1,10,resnet34_skips,imagenet_subset_kaggle,True
4,9.060000,finetuning,1,20,resnet34_skips,imagenet_subset_kaggle,True
...,...,...,...,...,...,...,...
115,28.231579,lwf,0,20,convnext_tiny,imagenet_subset_kaggle,True
116,43.680000,lwf,1,10,convnext_tiny,imagenet_subset_kaggle,True
117,29.389474,lwf,1,20,convnext_tiny,imagenet_subset_kaggle,True
118,42.260000,lwf,2,10,convnext_tiny,imagenet_subset_kaggle,True


In [145]:
df_noreg

,num_tasks,approach,is_regularized,avg_acc_tag,dataset,seed,network
0,5,finetuning,False,25.72,cifar100_fixed,1,convnext_tiny
2,5,replay,False,34.70,cifar100_fixed,1,convnext_tiny
4,5,ewc,False,29.55,cifar100_fixed,1,convnext_tiny
6,5,lwf,False,40.63,cifar100_fixed,1,convnext_tiny
8,10,finetuning,False,15.57,cifar100_fixed,1,convnext_tiny
...,...,...,...,...,...,...,...
6,10,lwf,False,36.60,imagenet_subset_kaggle,1,resnet34_skips
8,20,finetuning,False,5.78,imagenet_subset_kaggle,1,resnet34_skips
10,20,replay,False,34.06,imagenet_subset_kaggle,1,resnet34_skips
12,20,ewc,False,7.18,imagenet_subset_kaggle,1,resnet34_skips


In [146]:
merged = pd.concat([df_noreg, filtered_df], axis=0)

In [148]:
pivot_table = pd.pivot_table(
    merged[merged["network"].str.contains("resnet")],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    # aggfunc= lambda x: len(x),
    aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    fill_value=0,
)
pivot_table

dataset                   cifar100_fixed                           \
num_tasks                             5           10           20   
approach   is_regularized                                           
ewc        False              23.10±0.51  12.22±0.11    4.91±2.37   
           True               27.21±0.34  11.35±3.13    6.04±2.99   
finetuning False              20.55±0.35  11.54±0.10    5.03±0.61   
           True               23.62±0.21  11.97±1.82    6.39±2.79   
lwf        False              39.26±0.24  25.45±0.97   16.89±0.73   
           True               40.56±2.06  24.85±1.12  36.49±28.79   
replay     False              40.63±0.89  35.42±2.32   31.60±1.55   
           True               44.24±0.75  35.86±1.51   33.36±1.14   

dataset                   imagenet_subset_kaggle              
num_tasks                                     10          20  
approach   is_regularized                                     
ewc        False                      13.73±0.16   7.01±0.12  
           True                       21.09±0.54  11.83±0.06  
finetuning False                      11.52±0.12   6.08±0.24  
           True                       14.93±0.19   9.21±0.47  
lwf        False                      35.47±0.99  19.61±0.38  
           True                       35.37±0.53  19.31±0.05  
replay     False                      38.59±0.72  35.00±1.76  
           True                       39.96±0.46  36.31±1.18

In [149]:
pivot_table = pd.pivot_table(
    merged[merged["network"].str.contains("convnext")],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    # aggfunc= lambda x: len(x),
    aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    fill_value=0,
)
pivot_table

dataset                   cifar100_fixed                          \
num_tasks                             5           10          20   
approach   is_regularized                                          
ewc        False              29.57±0.03  16.92±0.55   8.87±0.53   
           True               29.05±1.21  17.02±0.69  10.26±0.07   
finetuning False              26.21±0.69  15.31±0.37   8.19±0.27   
           True               24.09±0.86  13.65±0.77   8.83±0.10   
lwf        False              40.70±0.09  29.83±0.21  23.01±0.87   
           True               43.99±0.48  30.55±0.78  21.72±0.25   
replay     False              34.53±0.25  28.59±0.13  25.42±0.25   
           True               33.87±0.24  27.08±0.39  25.14±0.23   

dataset                   imagenet_subset_kaggle              
num_tasks                                     10          20  
approach   is_regularized                                     
ewc        False                      21.65±0.70  10.81±0.39  
           True                       21.60±0.70  11.85±0.20  
finetuning False                      17.29±0.30   7.37±0.01  
           True                       16.23±0.69   8.00±0.61  
lwf        False                      43.73±0.46  29.60±0.71  
           True                       42.91±0.59  28.97±0.52  
replay     False                      34.55±0.86  29.78±0.74  
           True                       33.71±0.44  27.32±0.61